In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import cv2
import matplotlib.pyplot as plt
from  ultralytics import YOLO
import gradio as gr
import os
import numpy as np

In [ ]:
model = YOLO('/content/drive/MyDrive/dental/yolov8n dental.pt')

In [ ]:
def proceed_yolo_obj_detection(image):
    try:
        result = model(image[:, :, ::-1], save=True)
    except Exception as e:
        print(e)
        return

    class_names = result[0].names
    bg_colors = [(81, 16, 194), (194, 16, 75), (16, 78, 194), (156, 154, 145)]
    for i in range(len(result[0].boxes.data)):
        name = class_names[int(result[0].boxes.cls[i])]
        conf = round(float(result[0].boxes.conf[i]), 2)
        x1, y1, x2, y2 = [int(i) for i in result[0].boxes.xyxy[i]]

        text = f'{name} {conf}'
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        font_scale = 0.8
        font_thickness = 2
        text_color = (255, 255, 255)
        bg_color = bg_colors[int(result[0].boxes.cls[i])]
        (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)

        cv2.rectangle(image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), bg_color, -1)
        cv2.putText(image, text, (x1, y1-5), font, font_scale, text_color, font_thickness)
        cv2.rectangle(image, (x1, y1), (x2, y2), bg_color, 2)
    return image

article = "<h3>How to Use:</h3> " \
          "<ul><li>Open the Dental Radiogragy Website.</li> " \
          "<li>Enter Image in the provided input box.</li>" \
          "<li>Afterthat, Automatically Output is display in the provided output box.</li></ul>"



In [ ]:
def generate_patient_details(name, age, gender):
    # Create patient details string
    details = f"Patient Name: {name}\nPatient Age: {age} years\nPatient Gender: {', '.join(gender)}"

    # Save details to a text file
    file_path = "/tmp/patient_details.txt"
    with open(file_path, "w") as file:
        file.write(details)

    return file_path

def submit(name, age, gender):
    # Generate the file with patient details
    file_path = generate_patient_details(name, age, gender)

    return {
        submit_btn: gr.Button(visible=False),
        output_col: gr.Column(visible=True),
        patient_summary_box: f"Patient Name: {name}\n\nPatient Age: {age} years\n\nPatient Gender: {' '.join(gender)}",
        download_btn: gr.File(file_path, label="Download Patient Details")
    }

def clear_form(name, age, gender):
    return "", "", "", gr.Button(visible=True), gr.Column(visible=False), gr.File(visible=False)

# Define your Gradio interfaces
demo = gr.Interface(proceed_yolo_obj_detection, inputs=gr.Image(), outputs=gr.Image(),article=article, live=True)

with gr.Blocks() as demo1:
    name_box = gr.Textbox(label="Name", placeholder="Enter Full Name")
    age_box = gr.Number(label="Age", minimum=0, maximum=100)
    gender_box = gr.CheckboxGroup(["Male", "Female", "Other"], label="Gender")
    submit_btn = gr.Button("Submit")
    clear_btn = gr.Button("Clear")

    with gr.Column(visible=False) as output_col:
        patient_summary_box = gr.Textbox(label="Patient Summary")
        download_btn = gr.File(visible=True)

    submit_btn.click(
        submit,
        [name_box, age_box, gender_box],
        [submit_btn, patient_summary_box, output_col, clear_btn, download_btn]
    )

    clear_btn.click(
        clear_form,
        [name_box, age_box, gender_box],
        [name_box, age_box, gender_box, submit_btn, output_col, download_btn]
    )

# Create the tab interface
tab = gr.TabbedInterface(
    [demo1, demo],
    ["Patient Details", "Detection Page"],
    theme=gr.Theme.from_hub('HaleyCH/HaleyCH_Theme'),
    title='Dental Radiography'
)

tab.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


themes/theme_schema@0.0.1.json:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://056a0c174c044cba27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
